# Segmenting and Clustering Neighborhoods in Toronto
---

# Part 1
---

#### Adding required libraries and scraping the website

In [1]:
import pandas as pd
import requests as rq
!pip install BeautifulSoup4
from bs4 import BeautifulSoup

     |████████████████████████████████| 102kB 3.1MB/s ta 0:00:011


In [2]:
website_url = rq.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(website_url)

#### Reading the table from the Wikipedia page

In [3]:
wiki_table = soup.find('table',{'class':'wikitable sortable'})

#### extracting the row elements to a List

In [4]:
rows = wiki_table.findAll('tr')
wiki_table2 = []
for row in rows:
    dataset = row.find_all("td")
    wiki_table2.append(dataset)

### Extracting the data from the "td" tags to a clean List
### and handling 'Not assigned' values 

In [5]:
clean_table = []
brgh = ''
nbr = ''
for row in wiki_table2:
    try:
        if row[1].get_text() != 'Not assigned' and row[1].get_text() != '':
            brgh = row[1].get_text()
            if row[2].get_text().rstrip() != 'Not assigned':
                nbr = row[2].get_text().rstrip()
            else :
                nbr = brgh
            clean_table.append( [row[0].get_text(), brgh, nbr] )
    except:
        print('try/except to bypass empty lines')
        pass

try/except to bypass empty lines


## Creating a dataframe and loading from the list

In [6]:
df_postal = pd.DataFrame(clean_table) 

#### Adding column headings

In [7]:
df_postal.columns=['PostalCode', 'Borough', 'Neighborhood']

#### Grouping rows that have the same Postal Code

In [8]:
df_postal = df_postal.groupby('PostalCode').agg({ 'Borough':'first', 
                             'Neighborhood': ', '.join }).reset_index()
df_postal.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Getting dataframe shape

In [9]:
df_postal.shape

(103, 3)

---

# Part 2

In [10]:
df_geocod = pd.read_csv('http://cocl.us/Geospatial_data')

In [11]:
df_geocod.head(1)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353


In [12]:
df_geocod = df_geocod.rename(columns={'Postal Code':'PostalCode'})

#### Joining two dataframes

In [13]:
df_toronto_postal = pd.merge(df_postal, df_geocod , on = ['PostalCode'] )

In [14]:
df_toronto_postal.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


---

# Part 3

In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py37_1         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geogra

In [18]:
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [21]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [22]:


# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_postal['Latitude'], df_toronto_postal['Longitude'], df_toronto_postal['Borough'], df_toronto_postal['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto